# Human Capital Analytics and Reporting Case Study

In [1]:
# Import packages 
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from datetime import datetime, date

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
# Set display options
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 50)

In [3]:
# Read in the data and reate dataframe variables 
ra = pd.read_csv('Case_Study_HCAR_Research_2019_Recruiting_Activity_Data.csv')    # Recruiting Activity Dataframe
os = pd.read_csv('Case_Study_HCAR_Research_2019_Offer_Status_Data.csv')           # Offer Status Dataframe

In [4]:
'''
Check the dimensions of the datasets
Output is in the form (rows, columns)
'''
print(ra.shape)
print('')
print(os.shape)

(4972, 9)

(125, 3)


## Data Cleaning, Manipulation & Analysis - Recruiting Activity Dataframe

In [5]:
# Print information on the Recruiting Activity dataset
ra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4972 entries, 0 to 4971
Data columns (total 9 columns):
Candidate ID Number                  4959 non-null float64
Position Title                       4959 non-null object
Department                           4959 non-null object
Furthest Recruiting Stage Reached    4959 non-null object
Date of Application                  4959 non-null object
Application Source                   4959 non-null object
Highest Degree                       4959 non-null object
Years of Experience                  4959 non-null float64
Candidate Type                       4959 non-null object
dtypes: float64(2), object(7)
memory usage: 349.7+ KB


**There are null values in the dataset because the number of entries in each column doesn't match the total number of entries in the RangeIndex.**

In [6]:
# Check for null values
ra.isna().sum()

Candidate ID Number                  13
Position Title                       13
Department                           13
Furthest Recruiting Stage Reached    13
Date of Application                  13
Application Source                   13
Highest Degree                       13
Years of Experience                  13
Candidate Type                       13
dtype: int64

**There are 13 null values in each column.**

In [7]:
# Filter dataframe for all null values to examine them and determine what should be done with them
ra[ra.isna().any(axis=1)]

,Candidate ID Number,Position Title,Department,Furthest Recruiting Stage Reached,Date of Application,Application Source,Highest Degree,Years of Experience,Candidate Type
4959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4961,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Since the rows with null values dont contain any information they will be dropped from the dataframe.**

In [8]:
# Drop null values
ra.dropna(inplace=True)

In [9]:
# Check to see if all null values have been dropped
ra.isna().sum().sum()

0

In [10]:
# Analyze all columns except Candidate ID Number and Date of Application to see unique values in each column 
[print(ra[i].unique()) for i in ra.columns.drop(['Candidate ID Number', 'Date of Application'])]

['Associate Software Developer' 'Associate Relationship Manager'
 'Associate Product Manager' 'Finance Manager' 'Operations Coordinator'
 'Business Operations Manager' 'Sr. Business Analyst'
 'Sr. Product Manager' 'IT Analyst' 'Sr. Software Engineer'
 'Sr. Customer Service Operations Associate' 'UX Designer'
 'Financial Analyst' 'Account Executive' 'Operations Generalist']
['Engineering' 'Sales' 'Product' 'Finance' 'Operations' 'IT']
['Offer Sent' 'New Application' 'In-House Interview' 'Phone Screen']
['Campus Job Board' 'Campus Event' 'Advertisement' 'Career Fair' 'Website'
 'Agency' 'Outsourced' 'Internal Referral']
['PhD' 'Masters' 'Bachelors' 'JD' 'Phd']
[ 2.  0.  1.  3.  5.  7.  4. 15.  6. 13.  8. 10. 14. 12. 11.  9.]
['Campus' 'Experienced']


[None, None, None, None, None, None, None]

**In the highest degree column, there are 2 labels for PhD degrees. All values will be changed to `PhD` for consistency.**

In [11]:
# Change all values that are `Phd` into `PhD` 
ra.loc[ra['Highest Degree'] == 'Phd', 'Highest Degree'] = 'PhD'

In [12]:
# The change was successful
ra['Highest Degree'].value_counts()

Bachelors    2835
Masters      1208
PhD           909
JD              7
Name: Highest Degree, dtype: int64

In [13]:
# Check counts of all values in categorical columns 
print(ra['Department'].value_counts())
print('--------')
print(ra['Furthest Recruiting Stage Reached'].value_counts())
print('--------')
print(ra['Application Source'].value_counts())
print('--------')
print(ra['Highest Degree'].value_counts())
print('--------')
print(ra['Candidate Type'].value_counts())

Engineering    2113
Sales          1491
Product         710
Operations      289
Finance         197
IT              159
Name: Department, dtype: int64
--------
New Application       3364
Phone Screen           780
In-House Interview     691
Offer Sent             124
Name: Furthest Recruiting Stage Reached, dtype: int64
--------
Campus Job Board     1751
Career Fair          1044
Campus Event          623
Website               612
Advertisement         502
Agency                268
Outsourced            114
Internal Referral      45
Name: Application Source, dtype: int64
--------
Bachelors    2835
Masters      1208
PhD           909
JD              7
Name: Highest Degree, dtype: int64
--------
Campus         4131
Experienced     828
Name: Candidate Type, dtype: int64


In [14]:
# Function to change all categorical columns into numeric 
def to_numeric(df, col):
    df[col] = df[col].astype('category').cat.codes

In [15]:
# Apply function to categorical columns
to_numeric(df=ra, col='Department')
to_numeric(df=ra, col='Furthest Recruiting Stage Reached')
to_numeric(df=ra, col='Application Source')
to_numeric(df=ra, col='Highest Degree')
to_numeric(df=ra, col='Candidate Type')

In [16]:
# Check to see that the change was successful
ra.head()

,Candidate ID Number,Position Title,Department,Furthest Recruiting Stage Reached,Date of Application,Application Source,Highest Degree,Years of Experience,Candidate Type
0,2468.0,Associate Software Developer,0,2,11/1/18,3,3,2.0,0
1,2471.0,Associate Relationship Manager,5,2,12/13/18,2,2,0.0,0
2,2475.0,Associate Software Developer,0,2,12/21/18,0,2,0.0,0
3,2480.0,Associate Relationship Manager,5,2,12/25/18,2,2,0.0,0
4,2486.0,Associate Relationship Manager,5,2,12/6/18,2,2,1.0,0


In [17]:
'''
Dictionary to use as reference.
Numbers were automatically assigned in alphabetical order.
'''

department_dict = {0: 'Engineering', 1: 'Finanace', 2: 'IT', 3: 'Operations', 4: 'Product', 5: 'Sales'}

recruiting_stage_dict = {0: 'In-House Interview', 1: 'New Application', 2: 'Offer Sent', 3: 'Phone Screen'}

app_source_dict = {0: 'Advertisement', 1: 'Agency', 2: 'Campus Event', 3: 'Campus Job Board', 4: 'Career Fair',
                  5: 'Internal Referral', 6: 'Outsourced', 7: 'Website'}

highest_degree_dict = {0: 'Bachelors', 1: 'JD', 2: 'Masters', 3: 'PhD'}

candidate_type_dict = {0: 'Campus', 1: 'Experienced'}

In [18]:
# Change datatypes of Candidate ID Number and Years of Experience to integers
ra[['Candidate ID Number', 'Years of Experience']] = ra[['Candidate ID Number', 'Years of Experience']].applymap(np.int64)

In [19]:
'''
Now all columns are their appropriate datatypes.
This is important for modeling.
'''
ra.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4959 entries, 0 to 4958
Data columns (total 9 columns):
Candidate ID Number                  4959 non-null int64
Position Title                       4959 non-null object
Department                           4959 non-null int8
Furthest Recruiting Stage Reached    4959 non-null int8
Date of Application                  4959 non-null object
Application Source                   4959 non-null int8
Highest Degree                       4959 non-null int8
Years of Experience                  4959 non-null int64
Candidate Type                       4959 non-null int8
dtypes: int64(2), int8(5), object(2)
memory usage: 217.9+ KB


In [20]:
# Check for any duplicated rows
ra[ra.duplicated()]

,Candidate ID Number,Position Title,Department,Furthest Recruiting Stage Reached,Date of Application,Application Source,Highest Degree,Years of Experience,Candidate Type


**There are no duplicates of data.**

## Data Cleaning, Manipulation & Analysis - Offer Status DataFrame

In [21]:
# Print information on the Offer Status dataset
os.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 3 columns):
Candidate ID Number    125 non-null int64
Offer Date             125 non-null object
Offer Decision         125 non-null object
dtypes: int64(1), object(2)
memory usage: 3.0+ KB


**There are no null values in this dataset.**

In [22]:
# See what the data looks like
os.head()

,Candidate ID Number,Offer Date,Offer Decision
0,21031,11/15/16,Offer Accepted
1,26414,3/20/18,Offer Accepted
2,98318,2/27/17,Offer Accepted
3,35377,3/6/18,Offer Accepted
4,72131,11/28/17,Offer Accepted


In [23]:
# Check the number of each offer accepted, declined, or pending response 
os['Offer Decision'].value_counts()

Offer Accepted            68
Offer Declined            45
Offer Response Pending    12
Name: Offer Decision, dtype: int64

In [24]:
# Check rows where offer decision is something other than accept or decline
os[os['Offer Decision'] == 'Offer Response Pending']

,Candidate ID Number,Offer Date,Offer Decision
113,2520,1/5/19,Offer Response Pending
114,2471,1/31/19,Offer Response Pending
115,2501,1/26/19,Offer Response Pending
116,2531,2/15/19,Offer Response Pending
117,2493,1/26/19,Offer Response Pending
118,2543,2/6/19,Offer Response Pending
119,2556,12/28/18,Offer Response Pending
120,2475,2/5/19,Offer Response Pending
121,2480,2/9/19,Offer Response Pending
122,2486,1/17/19,Offer Response Pending


In [25]:
# Drop offer decision values that are not accept or decline because they are unnessecary for modeling
os.drop(os.index[113:], inplace=True)

In [26]:
# Binarize categorical column to be used for modeling
os['Offer Decision'] = os['Offer Decision'].map(lambda x: 1 if x == 'Offer Accepted' else 0)

In [27]:
# Check to see the drop and binarization was successful
os.tail()

,Candidate ID Number,Offer Date,Offer Decision
108,4545,11/4/18,0
109,3791,11/22/17,0
110,4811,11/8/17,0
111,4061,1/15/19,0
112,4005,1/9/17,0


In [28]:
os[os.duplicated()]

,Candidate ID Number,Offer Date,Offer Decision


**There are no duplicates of data.**

## Merge and Clean Dataframes 

In [29]:
# Join both dataframes on the Candidate ID Number
df_join = pd.merge(left=ra, right=os, how='outer')

In [30]:
# Create a new dataframe variable that contains the joined dataframed with null values dropped
model_df = df_join.dropna()

In [31]:
# Reset index of the dataframe and drop unnessecary columns
model_df.reset_index(inplace=True)
model_df.drop('index', axis=1, inplace=True)

**Now that the dataframe we will use to model is prepared, a new feature, `Date Range`, will be engineered which will be an important variable necessary for the model. This variable will indicate the number of days between the Date of Application and the Offer Date.**

In [32]:
# Convert the Date of Application and Offer Date into datetime objects
model_df['Date of Application'] = pd.to_datetime(model_df['Date of Application'])
model_df['Offer Date'] = pd.to_datetime(model_df['Offer Date'])

In [33]:
# Create new `Date Range` variable
model_df['Date Range'] = model_df['Offer Date'] - model_df['Date of Application']

In [34]:
# Check to see the new variable has been created
model_df.head()

,Candidate ID Number,Position Title,Department,Furthest Recruiting Stage Reached,Date of Application,Application Source,Highest Degree,Years of Experience,Candidate Type,Offer Date,Offer Decision,Date Range
0,2570,Associate Software Developer,0.0,2.0,2016-01-05,0.0,0.0,0.0,0.0,2016-02-14,1.0,40 days
1,2585,Associate Software Developer,0.0,2.0,2017-09-09,2.0,0.0,0.0,0.0,2017-10-20,1.0,41 days
2,2601,Associate Relationship Manager,5.0,2.0,2018-12-12,0.0,1.0,2.0,0.0,2019-01-11,1.0,30 days
3,2618,Associate Software Developer,0.0,2.0,2018-12-02,0.0,0.0,0.0,0.0,2019-01-04,1.0,33 days
4,2636,Associate Relationship Manager,5.0,2.0,2017-11-01,3.0,2.0,0.0,0.0,2017-12-05,1.0,34 days


In [35]:
# The float64 and Date Range variables need to be converted back to an integer in order to be used for modeling
model_df.dtypes

Candidate ID Number                            int64
Position Title                                object
Department                                   float64
Furthest Recruiting Stage Reached            float64
Date of Application                   datetime64[ns]
Application Source                           float64
Highest Degree                               float64
Years of Experience                          float64
Candidate Type                               float64
Offer Date                            datetime64[ns]
Offer Decision                               float64
Date Range                           timedelta64[ns]
dtype: object

In [36]:
# Show only the number of days, removing the word 'days'
model_df['Date Range'] = [i.days for i in model_df['Date Range']]

In [37]:
# Change float and Data Range columns to integer datatypes
model_df[['Department', 
          'Furthest Recruiting Stage Reached',
          'Application Source',
          'Highest Degree',
          'Years of Experience',
          'Candidate Type',
          'Offer Decision',
          'Date Range']] = model_df[['Department', 
                                     'Furthest Recruiting Stage Reached',
                                     'Application Source',
                                     'Highest Degree',
                                     'Years of Experience',
                                     'Candidate Type',
                                     'Offer Decision',
                                     'Date Range']].applymap(np.int64)

In [38]:
# All columns are now their correct datatype
model_df.dtypes

Candidate ID Number                           int64
Position Title                               object
Department                                    int64
Furthest Recruiting Stage Reached             int64
Date of Application                  datetime64[ns]
Application Source                            int64
Highest Degree                                int64
Years of Experience                           int64
Candidate Type                                int64
Offer Date                           datetime64[ns]
Offer Decision                                int64
Date Range                                    int64
dtype: object

In [39]:
# Inspect that the dataframe is clean for modeling
model_df.head()

,Candidate ID Number,Position Title,Department,Furthest Recruiting Stage Reached,Date of Application,Application Source,Highest Degree,Years of Experience,Candidate Type,Offer Date,Offer Decision,Date Range
0,2570,Associate Software Developer,0,2,2016-01-05,0,0,0,0,2016-02-14,1,40
1,2585,Associate Software Developer,0,2,2017-09-09,2,0,0,0,2017-10-20,1,41
2,2601,Associate Relationship Manager,5,2,2018-12-12,0,1,2,0,2019-01-11,1,30
3,2618,Associate Software Developer,0,2,2018-12-02,0,0,0,0,2019-01-04,1,33
4,2636,Associate Relationship Manager,5,2,2017-11-01,3,2,0,0,2017-12-05,1,34


In [52]:
# The average time in days our recruiting process is 
print(round(np.mean(model_df['Date Range'])), ' days')

50.0  days


## Modeling - Linear Regression

In [40]:
'''
Set up X and y variables which will be used for modeling.
A Linear Regression model will be built because it allows for interpretability of results and coefficients.
'''
features = model_df.columns.drop(['Candidate ID Number', 'Position Title', 'Date of Application',
                                  'Offer Date', 'Offer Decision'])
X = model_df[features]
y = model_df['Offer Decision']

In [41]:
'''
Set up training and testing sets.
Not many applications had an accompanying offer decision therefore the train and test sets will be small.
Although this is not ideal for modeling, for the purpose of this exercise it will do.
'''
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [42]:
# Build the model
lr = LinearRegression()

# Fit the model
lr.fit(X_train, y_train)

# Score the model
lr.score(X_test, y_test)

0.04205181229721955

**The model's R^2 value is .42 which means the explanatory power of the model is not strong, but for this study, the strength of the model is not the main focus. We want to see the degree of influence the Date Range variable has on the model so we care more about the variable's coefficients.**

In [43]:
# Examine the coefficients for each variable
lr.coef_

array([ 5.18209891e-03, -5.37764278e-17, -1.14343134e-02, -2.18350614e-03,
        1.89109537e-02, -4.35867855e-02, -6.86849683e-03])

In [44]:
# Set coefficients to a variable
coefficients = lr.coef_

# Create a dataframe to see the coefficients beside its respective variable
coef_df = pd.DataFrame(coefficients, columns=['coefficients'], index=features)
# Round all values to 2 decimal places
coef_df.round(2)

,coefficients
Department,0.01
Furthest Recruiting Stage Reached,-0.00
Application Source,-0.01
Highest Degree,-0.00
Years of Experience,0.02
Candidate Type,-0.04
Date Range,-0.01


**The dataframe shows that the Date Range has a negative coefficient. The coefficient can be interepreted as when days in the recruiting process increases by 1 day, the offer acceptance rate decreases by 0.01%**

## Modeling - Statsmodel

**Create null and alternative hypotheses for the statsmodel.**

$H_{0}$ (Null Hypothesis): There is no relationship between the longevity of the hiring process and offer acceptance rates.

$H_{A}$ (Alternative Hypothesis}: There is a relationship between the longevity of the hiring process and offer acceptance rates.

In [45]:
# Create features for the statsmodel
X = model_df[features]
y = model_df['Offer Decision']

X = sm.add_constant(X)

# Instantiate the model
model = sm.OLS(y, X)
# Fit the model
results = model.fit()
# Print a results summary for the model
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         Offer Decision   R-squared:                       0.109
Model:                            OLS   Adj. R-squared:                  0.058
Method:                 Least Squares   F-statistic:                     2.134
Date:                Thu, 21 Mar 2019   Prob (F-statistic):             0.0555
Time:                        10:26:38   Log-Likelihood:                -72.642
No. Observations:                 112   AIC:                             159.3
Df Residuals:                     105   BIC:                             178.3
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Department                            0.0147      0.024      0.625      0.533      -0.032       0.061
Furthest Recruiting Stage Reached     0.4886      0.076      6.460      0.000       0.339       0.639
Application Source                    0.0028      0.024      0.119      0.905      -0.044       0.050
Highest Degree                        0.0124      0.039      0.317      0.752      -0.065       0.090
Years of Experience                  -0.0017      0.025     -0.067      0.947      -0.052       0.048
Candidate Type                       -0.0100      0.221     -0.045      0.964      -0.449       0.429
Date Range                           -0.0085      0.002     -3.424      0.001      -0.013      -0.004
==============================================================================
Omnibus:                      114.748   Durbin-Watson:                   1.080
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                9.995
Skew:                          -0.231   Prob(JB):                      0.00675
Kurtosis:                       1.611   Cond. No.                         265.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**The p-value for the Date Range variable of 0.001 is lower than the alpha significance value (0.05) meaning that we can reject the null hypothesis and accept the alternative hypothesis that there is a relationship between the efficiency of the recruiting process and offer acceptance rates.**

## Results
### The results from the Linear Regression Model indicate that as the number of days in the recruiting process increases, the offer acceptance rate decreases by a small amount. The results summary from the Statsmodel indicates that there is a relationship between the length of the recruiting process and offer acceptance rates.

In [57]:
!jupyter nbconvert Human_Capital_Analytics_Case_Study.ipynb --to html

[NbConvertApp] Converting notebook Human_Capital_Analytics_Case_Study.ipynb to html
[NbConvertApp] Writing 356525 bytes to Human_Capital_Analytics_Case_Study.html
